# Overview
This is version 1.0 of the model. It is my better effort to create a model based on the specifications of evolving hockey in the following write up.
https://evolving-hockey.com/blog/a-new-expected-goals-model-for-predicting-goals-in-the-nhl/

In [1]:
#Import libraries
#used to import data
import pandas as pd
#XGB is the model that is used
import xgboost as xgb
from xgboost import cv
#Evaluates the model
from sklearn.metrics import log_loss

import time #just used to see how long the model takes to run

In [13]:
''' In their write up they say that for even strength, they used data from 10-11 to 16-17
    Then they tested it on 17-18 data that the model had not seen. 
    I'm using this strategy to see if my model performs roughly the same as theirs
'''
df_11 = pd.read_csv('data/pbp/ready_to_model/version_4/10_11_pbp_cleaned.csv')
df_12 = pd.read_csv('data/pbp/ready_to_model/version_4/11_12_pbp_cleaned.csv')
df_13 = pd.read_csv('data/pbp/ready_to_model/version_4/12_13_pbp_cleaned.csv')
df_14 = pd.read_csv('data/pbp/ready_to_model/version_4/13_14_pbp_cleaned.csv')
df_15 = pd.read_csv('data/pbp/ready_to_model/version_4/14_15_pbp_cleaned.csv')
df_16 = pd.read_csv('data/pbp/ready_to_model/version_4/15_16_pbp_cleaned.csv')
df_17 = pd.read_csv('data/pbp/ready_to_model/version_4/16_17_pbp_cleaned.csv')
df_18 = pd.read_csv('data/pbp/ready_to_model/version_4/17_18_pbp_cleaned.csv')
df = pd.concat([df_11,df_12,df_13,df_14,df_15,df_16,df_17])

In [8]:
def get_X_y(df):
    ''' This function takes in a dataframe and splits it into the X and y variables
    '''
    X = df.drop(['is_goal'], axis=1)
    y = df.is_goal
    
    return X,y

In [9]:
X_train, y_train = get_X_y(df)
X_test, y_test = get_X_y(df_18)

xgb_reg = xgb.XGBRegressor(objective='binary:logistic',
                           gamme = .12, 
                           eval_metric = 'logloss',
                           #eval_metric = 'auc', 
                           eta = .068,
                           subsample = .78,
                           colsample_bytree = .76,
                           min_child_weight = 9,
                           max_delta_step = 5,
                           nthread = 4)

start = time.time()
xgb_reg.fit(X_train, y_train)
print(start-time.time())

y_pred = xgb_reg.predict(X_test)
print(log_loss(y_test, y_pred))

[20:17:56] WARNING: ..\src\learner.cc:541: 
Parameters: { gamme } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


-25.249342441558838
0.19508612762239705


In [14]:
X_train, y_train = get_X_y(df)
X_test, y_test = get_X_y(df_18)

xgb_reg = xgb.XGBRegressor(objective='binary:logistic',
                           eval_metric = 'logloss',
                           #eval_metric = 'auc', 
                           eta = .068,
                           subsample = .78,
                           colsample_bytree = .76,
                           min_child_weight = 9,
                           max_delta_step = 5,
                           nthread = 4)

start = time.time()
xgb_reg.fit(X_train, y_train)
print(start-time.time())

y_pred = xgb_reg.predict(X_test)
print(log_loss(y_test, y_pred))

-23.02945876121521
0.19130366376542968


In [12]:
df.columns

Index(['Unnamed: 0', 'game_period', 'game_seconds', 'is_home', 'coords_x',
       'coords_y', 'coords_x_last', 'coords_y_last', 'shot_distance',
       'shot_angle', 'state_5v5', 'state_5v4', 'score_down_4', 'score_down_3',
       'score_down_2', 'score_down_1', 'score_even', 'score_up_1',
       'score_up_2', 'score_up_3', 'wrist_shot', 'deflected_shot', 'tip_shot',
       'slap_shot', 'backhand_shot', 'snap_shot', 'prior_shot_same',
       'prior_miss_same', 'prior_shot_opp', 'prior_miss_opp', 'prior_give_opp',
       'prior_take_opp', 'prior_hit_opp', 'prior_face', 'state_5v3',
       'score_up_4', 'wrap_shot', 'prior_block_same', 'prior_block_opp',
       'prior_give_same', 'prior_take_same', 'prior_hit_same', 'is_goal'],
      dtype='object')